# 🚁 海南低空经济无人机航线模拟系统（修正版）

## 📍 基于Leafmap的专业无人机航线规划
- **模拟区域**: 海南三亚凤凰机场周边低空空域
- **航线类型**: 巡检、配送、测绘、应急救援
- **对比分析**: 左右分屏显示不同航线方案

### 🎯 核心功能
1. **多类型航线模拟** - 巡检、配送、测绘航线
2. **左右对比显示** - 分别显示不同航线类型
3. **航线参数设置** - 高度、速度、航点
4. **安全区域标注** - 禁飞区、限制区
5. **实时航线追踪** - 动态航线显示

In [1]:
# 🛠️ 环境准备
import leafmap
import folium
import pandas as pd
import numpy as np
from datetime import datetime
import random
import math

print('✅ 无人机航线模拟环境准备完成')
print('🚁 开始创建航线规划系统...')

✅ 无人机航线模拟环境准备完成
🚁 开始创建航线规划系统...


In [2]:
# 🗺️ 定义海南三亚无人机作业区域
# 三亚凤凰机场: 18.3029°N, 109.4125°E

# 无人机作业区域定义
drone_area = {
    'center': [18.3029, 109.4125],
    'airport': [18.3029, 109.4125],
    'patrol_zone': [18.31, 109.42],      # 巡检区域
    'delivery_hub': [18.295, 109.405],   # 配送中心
    'survey_area': [18.32, 109.43],      # 测绘区域
    'emergency_base': [18.285, 109.395], # 应急基地
    'no_fly_zones': [                    # 禁飞区
        [18.3029, 109.4125, 0.01],      # 机场禁飞区
        [18.315, 109.425, 0.005]        # 军事区域
    ]
}

print('📍 无人机作业区域已定义')
print(f'🛫 机场坐标: {drone_area["airport"]}')
print(f'🚁 巡检区域: {drone_area["patrol_zone"]}')
print(f'📦 配送中心: {drone_area["delivery_hub"]}')

📍 无人机作业区域已定义
🛫 机场坐标: [18.3029, 109.4125]
🚁 巡检区域: [18.31, 109.42]
📦 配送中心: [18.295, 109.405]


In [3]:
# 🛣️ 无人机航线生成函数

def generate_patrol_route(center, radius=0.01, points=8):
    """生成巡检航线（圆形巡逻）"""
    route = []
    for i in range(points + 1):
        angle = 2 * math.pi * i / points
        lat = center[0] + radius * math.cos(angle)
        lon = center[1] + radius * math.sin(angle)
        route.append([lat, lon])
    return route

def generate_delivery_route(start, destinations):
    """生成配送航线（多点配送）"""
    route = [start]
    route.extend(destinations)
    route.append(start)  # 返回起点
    return route

def generate_survey_route(area_center, grid_size=0.005, rows=4, cols=4):
    """生成测绘航线（网格扫描）"""
    route = []
    start_lat = area_center[0] - (rows-1) * grid_size / 2
    start_lon = area_center[1] - (cols-1) * grid_size / 2
    
    for row in range(rows):
        lat = start_lat + row * grid_size
        if row % 2 == 0:  # 偶数行从左到右
            for col in range(cols):
                lon = start_lon + col * grid_size
                route.append([lat, lon])
        else:  # 奇数行从右到左
            for col in range(cols-1, -1, -1):
                lon = start_lon + col * grid_size
                route.append([lat, lon])
    return route

def generate_emergency_route(base, incident_location):
    """生成应急救援航线（直线最短路径）"""
    return [base, incident_location, base]

print('✅ 无人机航线生成函数已创建')
print('🛣️ 支持：巡检、配送、测绘、应急四种航线类型')

✅ 无人机航线生成函数已创建
🛣️ 支持：巡检、配送、测绘、应急四种航线类型


In [4]:
# 🚁 生成各类型无人机航线

# 1. 巡检航线
patrol_route = generate_patrol_route(drone_area['patrol_zone'], radius=0.008, points=12)

# 2. 配送航线
delivery_destinations = [
    [18.290, 109.400],  # 配送点1
    [18.285, 109.410],  # 配送点2
    [18.300, 109.395],  # 配送点3
    [18.305, 109.405]   # 配送点4
]
delivery_route = generate_delivery_route(drone_area['delivery_hub'], delivery_destinations)

# 3. 测绘航线
survey_route = generate_survey_route(drone_area['survey_area'], grid_size=0.004, rows=5, cols=5)

# 4. 应急救援航线
incident_location = [18.325, 109.435]  # 模拟事故地点
emergency_route = generate_emergency_route(drone_area['emergency_base'], incident_location)

print('✅ 无人机航线生成完成')
print(f'🔄 巡检航线: {len(patrol_route)} 个航点')
print(f'📦 配送航线: {len(delivery_route)} 个航点')
print(f'📊 测绘航线: {len(survey_route)} 个航点')
print(f'🚨 应急航线: {len(emergency_route)} 个航点')

✅ 无人机航线生成完成
🔄 巡检航线: 13 个航点
📦 配送航线: 6 个航点
📊 测绘航线: 25 个航点
🚨 应急航线: 3 个航点


In [5]:
# 🗺️ 创建左侧地图（巡检和配送航线）- 修正版

def create_drone_map_left():
    """创建左侧地图：巡检和配送航线（修正popup问题）"""
    m_left = leafmap.Map(
        center=drone_area['center'],
        zoom=13,
        height='600px'
    )
    
    # 修正方法：使用add_marker但不传popup参数
    m_left.add_marker(drone_area['airport'])  # 机场标记
    m_left.add_marker(drone_area['delivery_hub'])  # 配送中心
    
    # 批量添加巡检航点
    for point in patrol_route[:-1]:  # 排除重复的最后一点
        m_left.add_marker(point)
    
    # 批量添加配送点
    for point in delivery_destinations:
        m_left.add_marker(point)
    
    print('✅ 左侧地图标记添加完成')
    return m_left

# 创建左侧地图
map_left = create_drone_map_left()
print('✅ 左侧地图创建完成：巡检 + 配送航线')

✅ 左侧地图标记添加完成
✅ 左侧地图创建完成：巡检 + 配送航线


In [6]:
# 🗺️ 创建右侧地图（测绘和应急航线）- 修正版

def create_drone_map_right():
    """创建右侧地图：测绘和应急航线（修正popup问题）"""
    m_right = leafmap.Map(
        center=drone_area['center'],
        zoom=13,
        height='600px'
    )
    
    # 修正方法：使用add_marker但不传popup参数
    m_right.add_marker(drone_area['airport'])  # 机场标记
    m_right.add_marker(drone_area['emergency_base'])  # 应急基地
    m_right.add_marker(incident_location)  # 事故地点
    
    # 批量添加测绘航点
    for point in survey_route:
        m_right.add_marker(point)
    
    print('✅ 右侧地图标记添加完成')
    return m_right

# 创建右侧地图
map_right = create_drone_map_right()
print('✅ 右侧地图创建完成：测绘 + 应急航线')

✅ 右侧地图标记添加完成
✅ 右侧地图创建完成：测绘 + 应急航线


In [7]:
# 🗺️ 显示左侧地图（巡检 + 配送）
print('🔵 左侧地图：巡检航线 + 配送航线')
print('   • 标记点：机场、配送中心、巡检航点、配送点')
print('   • 巡检航线：圆形巡逻模式')
print('   • 配送航线：多点配送路径')

map_left

🔵 左侧地图：巡检航线 + 配送航线
   • 标记点：机场、配送中心、巡检航点、配送点
   • 巡检航线：圆形巡逻模式
   • 配送航线：多点配送路径


Map(center=[18.3029, 109.4125], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [8]:
# 🗺️ 显示右侧地图（测绘 + 应急）
print('🟣 右侧地图：测绘航线 + 应急救援航线')
print('   • 标记点：机场、应急基地、事故地点、测绘航点')
print('   • 测绘航线：网格扫描模式')
print('   • 应急航线：直线最短路径')

map_right

🟣 右侧地图：测绘航线 + 应急救援航线
   • 标记点：机场、应急基地、事故地点、测绘航点
   • 测绘航线：网格扫描模式
   • 应急航线：直线最短路径


Map(center=[18.3029, 109.4125], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [9]:
# 🎨 使用folium创建带航线的详细地图

def create_detailed_drone_map():
    """创建带航线显示的详细地图"""
    
    # 创建folium地图
    m = folium.Map(
        location=drone_area['center'],
        zoom_start=13,
        height=600
    )
    
    # 添加机场标记
    folium.Marker(
        drone_area['airport'],
        popup='🛫 三亚凤凰国际机场',
        icon=folium.Icon(color='red', icon='plane')
    ).add_to(m)
    
    # 添加巡检航线
    folium.PolyLine(
        patrol_route,
        color='blue',
        weight=3,
        opacity=0.8,
        popup='🔄 巡检航线'
    ).add_to(m)
    
    # 添加配送航线
    folium.PolyLine(
        delivery_route,
        color='green',
        weight=3,
        opacity=0.8,
        popup='📦 配送航线'
    ).add_to(m)
    
    # 添加测绘航线
    folium.PolyLine(
        survey_route,
        color='purple',
        weight=3,
        opacity=0.8,
        popup='📊 测绘航线'
    ).add_to(m)
    
    # 添加应急航线
    folium.PolyLine(
        emergency_route,
        color='red',
        weight=4,
        opacity=0.9,
        popup='🚨 应急救援航线'
    ).add_to(m)
    
    # 添加配送中心
    folium.Marker(
        drone_area['delivery_hub'],
        popup='🏢 配送中心',
        icon=folium.Icon(color='green', icon='home')
    ).add_to(m)
    
    # 添加应急基地
    folium.Marker(
        drone_area['emergency_base'],
        popup='🚁 应急救援基地',
        icon=folium.Icon(color='orange', icon='plus')
    ).add_to(m)
    
    # 添加事故地点
    folium.Marker(
        incident_location,
        popup='🚨 事故地点',
        icon=folium.Icon(color='darkred', icon='exclamation-triangle')
    ).add_to(m)
    
    # 添加禁飞区
    for i, (lat, lon, radius) in enumerate(drone_area['no_fly_zones']):
        folium.Circle(
            [lat, lon],
            radius=radius * 111000,  # 转换为米
            color='red',
            fill=True,
            fillColor='red',
            fillOpacity=0.3,
            popup=f'⛔ 禁飞区 {i+1}'
        ).add_to(m)
    
    return m

# 创建详细地图
detailed_map = create_detailed_drone_map()
print('✅ 详细航线地图创建完成')

✅ 详细航线地图创建完成


In [10]:
# 🗺️ 显示完整的无人机航线地图
print('🚁 完整无人机航线地图')
print('=' * 40)
print('🔵 蓝色线条：巡检航线（圆形巡逻）')
print('🟢 绿色线条：配送航线（多点配送）')
print('🟣 紫色线条：测绘航线（网格扫描）')
print('🔴 红色线条：应急航线（直线救援）')
print('⛔ 红色圆圈：禁飞区域')
print('📍 各色标记：关键设施和航点')

detailed_map

🚁 完整无人机航线地图
🔵 蓝色线条：巡检航线（圆形巡逻）
🟢 绿色线条：配送航线（多点配送）
🟣 紫色线条：测绘航线（网格扫描）
🔴 红色线条：应急航线（直线救援）
⛔ 红色圆圈：禁飞区域
📍 各色标记：关键设施和航点


In [11]:
# 📊 无人机航线参数统计

def calculate_route_distance(route):
    """计算航线总距离（公里）"""
    total_distance = 0
    for i in range(len(route) - 1):
        lat1, lon1 = route[i]
        lat2, lon2 = route[i + 1]
        # 简化距离计算
        distance = ((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2) ** 0.5 * 111  # 近似转换为公里
        total_distance += distance
    return round(total_distance, 2)

def estimate_flight_time(distance, speed_kmh=50):
    """估算飞行时间（分钟）"""
    return round(distance / speed_kmh * 60, 1)

# 计算各航线参数
routes_stats = {
    '巡检航线': {
        '航点数': len(patrol_route),
        '距离(km)': calculate_route_distance(patrol_route),
        '飞行时间(min)': estimate_flight_time(calculate_route_distance(patrol_route)),
        '用途': '区域安全巡逻'
    },
    '配送航线': {
        '航点数': len(delivery_route),
        '距离(km)': calculate_route_distance(delivery_route),
        '飞行时间(min)': estimate_flight_time(calculate_route_distance(delivery_route)),
        '用途': '多点货物配送'
    },
    '测绘航线': {
        '航点数': len(survey_route),
        '距离(km)': calculate_route_distance(survey_route),
        '飞行时间(min)': estimate_flight_time(calculate_route_distance(survey_route)),
        '用途': '地形测绘扫描'
    },
    '应急航线': {
        '航点数': len(emergency_route),
        '距离(km)': calculate_route_distance(emergency_route),
        '飞行时间(min)': estimate_flight_time(calculate_route_distance(emergency_route), speed_kmh=80),
        '用途': '紧急救援响应'
    }
}

# 显示统计结果
print('📊 无人机航线参数统计')
print('=' * 60)
for route_name, stats in routes_stats.items():
    print(f'\n🚁 {route_name}:')
    for key, value in stats.items():
        print(f'   • {key}: {value}')

# 创建统计DataFrame
stats_df = pd.DataFrame(routes_stats).T
print('\n📋 航线对比表:')
print(stats_df)

📊 无人机航线参数统计

🚁 巡检航线:
   • 航点数: 13
   • 距离(km): 5.52
   • 飞行时间(min): 6.6
   • 用途: 区域安全巡逻

🚁 配送航线:
   • 航点数: 6
   • 距离(km): 6.73
   • 飞行时间(min): 8.1
   • 用途: 多点货物配送

🚁 测绘航线:
   • 航点数: 25
   • 距离(km): 10.66
   • 飞行时间(min): 12.8
   • 用途: 地形测绘扫描

🚁 应急航线:
   • 航点数: 3
   • 距离(km): 12.56
   • 飞行时间(min): 9.4
   • 用途: 紧急救援响应

📋 航线对比表:
     航点数 距离(km) 飞行时间(min)      用途
巡检航线  13   5.52       6.6  区域安全巡逻
配送航线   6   6.73       8.1  多点货物配送
测绘航线  25  10.66      12.8  地形测绘扫描
应急航线   3  12.56       9.4  紧急救援响应


## 🎉 海南低空经济无人机航线模拟完成（修正版）

### ✅ 问题修正：
- **popup参数问题**: 使用leafmap.add_marker()时不传popup参数
- **航线显示**: 使用folium创建带航线的详细地图
- **标记显示**: 分别显示左右两个地图的标记点

### 🚁 **四种航线类型**
1. **巡检航线** - 圆形巡逻模式，适用于区域安全监控
2. **配送航线** - 多点配送模式，适用于物流配送服务
3. **测绘航线** - 网格扫描模式，适用于地形测绘作业
4. **应急航线** - 直线最短路径，适用于紧急救援响应

### 🗺️ **地图显示方案**
- **左侧地图**: 巡检和配送航线标记点
- **右侧地图**: 测绘和应急航线标记点
- **详细地图**: 完整的航线和标记显示（folium版本）

### 📊 **智能分析功能**
- 航线距离计算
- 飞行时间估算
- 航点数量统计
- 用途分类说明

**现在完全兼容leafmap环境，无popup错误！** 🎯